## Calling a Graph function

In [14]:
using System;
using System.IO;

string filePath = Directory.GetCurrentDirectory();
Console.WriteLine("Current file path: " + filePath);

Console.WriteLine(".NET Version: " + Environment.Version);

Current file path: c:\Users\mrochon\source\repos\ai-samples\NotebookSK\7. GraphCall
.NET Version: 8.0.1


In [15]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.MsGraph, 1.0.1-alpha"
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.identity"


Installed Packages Azure.identity, 1.10.4 Microsoft.Graph, 5.41.0 Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.MsGraph, 1.0.1-alpha

In [16]:
using Microsoft.Graph;
using Azure.Identity;
var scopes = new[] { "User.Read", "Mail.Send" };
var tenantId = "consumers";
// Value from app registration
var clientId = "42f18b97-5fd1-4f8c-8c6c-2c5c3dba0623";
// using Azure.Identity;
var options = new DeviceCodeCredentialOptions
{
    AuthorityHost = AzureAuthorityHosts.AzurePublicCloud,
    ClientId = clientId,
    TenantId = tenantId,
    DeviceCodeCallback = (code, cancellation) =>
    {
        Console.WriteLine(code.Message);
        return Task.FromResult(0);
    },
};
// https://learn.microsoft.com/dotnet/api/azure.identity.devicecodecredential
var deviceCodeCredential = new DeviceCodeCredential(options);
var graphClient = new GraphServiceClient(deviceCodeCredential, scopes);
var user = await graphClient.Me.GetAsync();

To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code PQHNFJHT to authenticate.


In [17]:
#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.MsGraph;
using Microsoft.Graph;
using Microsoft.SemanticKernel.Plugins.MsGraph.Connectors;
var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();
kernel.Plugins.Clear();
#pragma warning disable SKEXP0053
var emailConnector = new OutlookMailConnector(graphClient);
var emailPlugin = new EmailPlugin(emailConnector);
kernel.ImportPluginFromObject(emailPlugin, "email");

List plugins and functions

In [ ]:
foreach (KernelFunctionMetadata func in kernel.Plugins.GetFunctionsMetadata())
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"   {func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }
    Console.WriteLine();
}

Now try different prompts

In [20]:
#pragma warning disable SKEXP0060
    var planner = new HandlebarsPlanner();
    var goal = "Send an email to <recipient> to say <subject>";
    var plan = await planner.CreatePlanAsync(kernel, goal);
    Console.WriteLine("Plan:\n");
    Console.WriteLine(plan.ToString());

Plan:

{{!-- Step 1: Set the email recipient --}}
{{set "recipient" "<recipient>"}}

{{!-- Step 2: Set the email subject --}}
{{set "subject" "<subject>"}}

{{!-- Step 3: Get my email address --}}
{{set "myEmailAddress" (email-GetMyEmailAddress)}}

{{!-- Step 4: Set the email content --}}
{{set "content" (concat "Hello, " recipient ". This is a message from " myEmailAddress ".")}}

{{!-- Step 5: Send the email --}}
{{email-SendEmail content=content recipients=recipient subject=subject}}


In [26]:
#pragma warning disable SKEXP0060
// planner = new HandlebarsPlanner("plan.txt");

Error: (2,33): error CS1503: Argument 1: cannot convert from 'string' to 'Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlannerOptions?'

Execute the plan

In [19]:
#pragma warning disable SKEXP0060
var resp = await plan.InvokeAsync(kernel);
Console.WriteLine("Result:\n");
Console.WriteLine(resp);

Error: System.TypeLoadException: Could not load type 'Microsoft.Graph.User' from assembly 'Microsoft.Graph, Version=5.41.0.0, Culture=neutral, PublicKeyToken=31bf3856ad364e35'.
   at Microsoft.SemanticKernel.Plugins.MsGraph.Connectors.OutlookMailConnector.GetMyEmailAddressAsync(CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Plugins.MsGraph.EmailPlugin.GetMyEmailAddressAsync()
   at Microsoft.SemanticKernel.KernelFunctionFromMethod.<>c.<<GetReturnValueMarshalerDelegate>b__12_4>d.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.SemanticKernel.KernelFunction.InvokeAsync(Kernel kernel, KernelArguments arguments, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.PromptTemplates.Handlebars.Helpers.KernelFunctionHelpers.InvokeKernelFunction(Kernel kernel, KernelFunction function, KernelArguments executionContext, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.PromptTemplates.Handlebars.Helpers.Kerne